In [2]:
# default inputs
import tellurium as te
import matplotlib.pyplot as plt
%config Completer.use_jedi = False
showOriginalModelString = True

In [3]:
# target outputs. The original and edited models should produce same outcomes for these outputs
targets = ['IKB', 'NFKB', 'pIKK', 'TAK1']

In [6]:
if showOriginalModelString:
    model = te.loadSBMLModel("Zhao_2021.xml")
    modelStr = model.getAntimony()
    print(modelStr)
    

// Created by libAntimony v2.12.0
model *pad_mac()

  // Compartments and Species:
  compartment unnamed;
  species deg in unnamed, V165a in unnamed, IFNG in unnamed, IFNGR_JAK in unnamed;
  species IFNG_R_JAK in unnamed, pIFNG_R_JAK in unnamed, STAT1 in unnamed;
  species pIFNG_R_JAK_STAT1 in unnamed, pSTAT1 in unnamed, pSTAT1D in unnamed;
  species pSTAT1D_n in unnamed, STAT1D_n in unnamed, STAT1_n in unnamed, $SOCS1_prod in unnamed;
  species SOCS1 in unnamed, IFNG_R_JAK_SOCS1 in unnamed, IFNG_R_lyso in unnamed;
  species SOCS1_JAK in unnamed, IRF9 in unnamed, IRF9_n in unnamed, pSTAT1D_IRF9 in unnamed;
  species pSTAT1D_IRF9_n in unnamed, $CXCL10_prod in unnamed, SOCS3 in unnamed;
  species IFNG_R_JAK_SOCS3 in unnamed, SOCS3_JAK in unnamed, $SOCS3_prod in unnamed;
  species IL4 in unnamed, IL4R_JAK in unnamed, IL4_R_JAK in unnamed, pIL4_R_JAK in unnamed;
  species STAT6 in unnamed, pIL4_R_JAK_STAT6 in unnamed, pSTAT6 in unnamed;
  species pSTAT6D in unnamed, pSTAT6D_n in unnamed, S

In [5]:
def run(model,selections, duration):
    model.integrator.absolute_tolerance = 1e-9
    model.integrator.relatice_tolerance = 1e-9
    results = model.simulate(start = 0, end = duration,steps = duration,
                             selections = selections)
    return results
def modify(modelStr,replacements):
    for key,value in replacements.items():
        modelStr = modelStr.replace(key,value)
    return modelStr
def merge(modelStr,sub_model):
    rr = modelStr.split('\nend\n\npad_mac')
    combined = rr[0]+sub_model+'\nend\n\npad_mac is "pad mac"\n'
    return combined

In [24]:
replacements = {
    '$ikk_prod => IKK; k191*ikk_prod':'NEMO_IKK + Mg => Mg_NEMO + IKK; k302*Mg*NEMO_IKK - k303*IKK - k304*Mg_NEMO',
    'IKK + pTAK1 => pIKK + pTAK1; k189*IKK*pTAK1^2': 'IKK + pTAK1 => pIKK + pTAK1; k305*IKK*pTAK1^2',
    'IKK + O2_aPHD => deg + O2_aPHD;  k192*IKK*O2_aPHD^2': 'IKK + O2_aPHD => deg + O2_aPHD; k306*IKK*O2_aPHD^2',
    'pIKK => IKK; k190*pIKK': 'pIKK => IKK; k307*pIKK'
}
extra_additions = """
    Mg_e = 100000; # 
    Mg = 100000 ; # internal/physiological Mg
    $Mg_e -> Mg; k301*(Mg_e - Mg) ; # Mg diffuses from extra- to intracellular space through345 TRPM7  
    NEMO_IKK_prod = 1;
    $NEMO_IKK_prod => NEMO_IKK; k308; 
    NEMO_IKK = 100000;
    k301 = .1;  k302 = 100; k303 = 0.1; k304 = 0.1; k305 = 5e-08; k306 = 5.2e-10; k307 = 1; k308 = 100;

"""
originalModel = te.loadSBMLModel("Zhao.xml")
originalModelStr = originalModel.getAntimony ()
modified_model = modify(originalModelStr,replacements)
combined = merge(modified_model,extra_additions)
combined = te.loada(combined)
combined.exportToSBML('Mg_M.xml')
# selections = ['TIME','Mg_e','Mg']
selections = ['TIME','pIKK']
results = run(combined,selections,duration = 1500)
# combined.plot()
print(results['pIKK'])

[8.70000000e+01 2.00510876e+05 2.00610885e+05 ... 3.50182881e+05
 3.50282876e+05 3.50382870e+05]


In [6]:
# test_model = """
#     Mg_e = 1; # 
#     Mg = 1 ; # internal/physiological Mg
#     k301 = .1 ; $Mg_e -> Mg; k301*(Mg_e - Mg) ; # Mg diffuses from extra- to intracellular space through345 TRPM7  
#     Mg + NEMO_IKK -> Mg_NEMO + IKK; k302*Mg*NEMO_IKK - k303*IKK - k304*Mg_NEMO ;
#     k302 = 0.1 ; k303 = 0.1; k304 = 0.3 ; NEMO_IKK = .5;
#     NEMO_IKK_prod = 1;
#     $NEMO_IKK_prod -> NEMO_IKK; 0.1*NEMO_IKK_prod-.1*NEMO_IKK;
#     Mg_NEMO->deg; .002*Mg_NEMO;
    
# """
# test_model = te.loada(test_model)
# # selections = ['TIME','NFKB','NFKB_n']
# selections = ['TIME','Mg','NEMO_IKK','Mg_NEMO','IKK']
# test_model.simulate(0,1500,1500,selections= selections)
# test_model.plot()